In [1]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import nltk
import os
import re

## Reading datasets

In [2]:
train=pd.read_csv(r"C:\Users\ishita\Downloads\fake-news\train.csv")
test=pd.read_csv(r"C:\Users\ishita\Downloads\fake-news\test.csv")

In [3]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [5]:
train['text'][1]

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [6]:
print(train.shape)
print(test.shape)

(20800, 5)
(5200, 4)


In [7]:
#checking null values
print(train.isnull().sum())
print("*"*10)
print(test.isnull().sum())

id           0
title      558
author    1957
text        39
label        0
dtype: int64
**********
id          0
title     122
author    503
text        7
dtype: int64


In [8]:
#need to full null values
train=train.fillna(' ')
test=test.fillna(' ')

In [9]:
test['total']=test['title']+' '+test['author']+test['text']
train['total']=train['title']+' '+train['author']+train['text']

In [10]:
train['total'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It Darrell LucusHouse Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of

### Cleaning and preprocessing

##### Now I would transform the  data by removing stopwords, using regular expressions module to accept only letters, tokenize those words and then make all the words lower case for consistency

In [11]:
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [14]:
from nltk.stem import WordNetLemmatizer

In [15]:
total=[]
stop_words=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()
for words in train['total']:
    only_letters=re.sub("[^a-zA-Z]"," ", words) #cleaning
    tokens = nltk.word_tokenize(only_letters) #tokenize the sentences
    lower_case=[word.lower() for word in tokens] #convert all letters to lower case
    filtered_result= [lemmatizer.lemmatize(word) for word  in lower_case if not word in stop_words]#remove stopwords from the text
    total.append(' '.join(filtered_result))

In [16]:
total[6]

'life life luxury elton john favorite shark picture stare long transcontinental flight ever wonder britain iconic pop pianist get long flight six picture shark sir elton john like stare pas time airplane shark swimming sit next elton john flight see man living lap luxury immediately tape photo shark darting ocean window stare throughout trip platinum selling artist known pull window shade make shark disappear quickly retract mark animal triumphant return often repeat act dozen time per flight great white shark leaping air international pop sensation worth estimated million people go extra mile make happy flight crew instructed place photo breaching great white elton john seat back pocket even board plane five time grammy winner love pull image leisure stare flying sometimes holding mere inch away face fixates upon shark beady eye shark prowl rumor elton john first acquired shark photo flying london perform candle wind princess diana funeral staple travel routine ever since flown music 

In [23]:
train['total']=total

In [48]:
train=train[['total','label']]
train

,total,label
0,house dem aide even see comey letter jason cha...,1
1,flynn hillary clinton big woman campus breitba...,0
2,truth might get fired consortiumnews comwhy tr...,1
3,civilian killed single u airstrike identified ...,1
4,iranian woman jailed fictional unpublished sto...,1
...,...,...
20795,rapper trump poster child white supremacy jero...,0
20796,n f l playoff schedule matchup odds new york t...,0
20797,macy said receive takeover approach hudson bay...,0
20798,nato russia hold parallel exercise balkan alex...,1


In [49]:
X_train = train['total']
Y_train = train['label']

In [50]:
#TFiDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_v= TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X=tfidf_v.fit_transform(X_train).toarray()

In [51]:
X.shape

(20800, 5000)

In [52]:
##Divide the dataset into train and test
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y_train,test_size=0.33,random_state=0)

In [53]:
print(X_train.shape,X_test.shape)

(13936, 5000) (6864, 5000)


In [54]:
tfidf_v.get_feature_names()[:20]

['aaron',
 'abandon',
 'abandoned',
 'abc',
 'abe',
 'abedin',
 'ability',
 'able',
 'abortion',
 'abroad',
 'absence',
 'absolute',
 'absolutely',
 'abu',
 'abuse',
 'academic',
 'academy',
 'accept',
 'acceptable',
 'acceptance']

In [55]:
tfidf_v.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 5000,
 'min_df': 1,
 'ngram_range': (1, 3),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

In [56]:
count_df=pd.DataFrame(X_train,columns=tfidf_v.get_feature_names())

In [57]:
count_df.head()

,aaron,abandon,abandoned,abc,abe,abedin,ability,able,abortion,abroad,...,young,young people,younger,youth,youtube,zero,zika,zionist,zone,zu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.019835,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in iter.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

## Modelling

###### Logistic Regression

In [61]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(C=1e5)
logreg.fit(X_train,Y_train)
pred=logreg.predict(X_test)
print('Accuracy of Logistic classifier on training set: {:.2f}'.format(logreg.score(X_train,Y_train)))
print('Accuracy of Logistic classifier on test set:{:.2f}'.format(logreg.score(X_test,Y_test)))
cm=confusion_matrix(Y_test,pred)
cm

c:\users\ishita\anaconda3\envs\fakenews\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy of Logistic classifier on training set: 1.00
Accuracy of Logistic classifier on test set:0.96


array([[3246,  144],
       [ 113, 3361]], dtype=int64)

#### MultinomialNB

In [63]:
from sklearn.naive_bayes import MultinomialNB

In [64]:

NB = MultinomialNB()
NB.fit(X_train, Y_train)
pred = NB.predict(X_test)
print('Accuracy of NB  classifier on training set: {:.2f}'
     .format(NB.score(X_train, Y_train)))
print('Accuracy of NB classifier on test set: {:.2f}'
     .format(NB.score(X_test, Y_test)))
cm = confusion_matrix(Y_test, pred)
cm

Accuracy of NB  classifier on training set: 0.91
Accuracy of NB classifier on test set: 0.90


array([[3176,  214],
       [ 502, 2972]], dtype=int64)

##### Passive Aggressive Classifier

In [69]:
from sklearn.linear_model import PassiveAggressiveClassifier
linear_clf=PassiveAggressiveClassifier()
linear_clf.fit(X_train,Y_train)
pred=linear_clf.predict(X_test)
print('Accuracy of Linear_clf classifier on training set: {:.2f}'
     .format(NB.score(X_train, Y_train)))
print('Accuracy of Linear_clf classifier on test set: {:.2f}'
     .format(NB.score(X_test, Y_test)))
cm = confusion_matrix(Y_test, pred)
cm

Accuracy of Linear_clf classifier on training set: 0.91
Accuracy of Linear_clf classifier on test set: 0.90


array([[3256,  134],
       [ 107, 3367]], dtype=int64)

### Model save

In [70]:
import pickle
file=open('fake_news_detection.pkl','wb')
#dump information to that file
model=pickle.dump(logreg,file)